In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
class BookSelection:
    if __name__ == "__main__":
        def __init__(self, url, search_keywords):
            self.url = url
            self.search_keywords = search_keywords

        def set_drivers(self):
            try:
                service = Service(executable_path=ChromeDriverManager().install())
                options = Options()
                # 참고 = https://codingstep.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%ED%81%AC%EB%A1%AC-%EC%98%B5%EC%85%98-python-selenium-chrome-optionsFeat-headless-window-size-user-agent-etc
                #지정한 user-agent로 설정합니다.
#                 user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
#                 options.add_argument('user-agent=' + user_agent)
#                 options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
#                 options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
#                 options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
#                 options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
#                 options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
#                 options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
#                 options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
                driver = webdriver.Chrome(service=service, options=options)  ##버전에 상관없이 현재 OS에 설치된 크롬브라우저 사용
                return driver
            except Exception as e:
                print(e)
        
        def get_url(self, driver):
            try:
                driver.get(self.url)
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_searchKeywords(self, driver):
            try:
                search_input = driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/div/input")
                search_input.send_keys(self.search_keywords)
                driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/input").click()
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
        
        def get_currentPage(self, driver):
            try:
                currentPage = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/ul/li/strong").text
                return currentPage
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_totalPage(self, driver):
            try:
                totPageCnt = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/span/span").text
                return totPageCnt
            except Exception as e:
                print(e)
                driver.quit()
        
        def get_searchList(self, driver):
            try:
                searchList = driver.find_element(By.ID, "search_list")
#                 searchList = driver.find_element(By.ID, "search_gallery")
                return searchList
            except Exception as e:
                print(e)
                driver.quit()
                
                
        def crawling_detailPage(self, driver, searchList):
            try:
                # searchListLength = len(searchList.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a")) ## 펼쳐보기에서 title Link가져오기
                searchListLength = len(searchList.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[2]/table/tbody/tr/td/div/div[4]/a")) ## 간략보기에서 title Link가져오기
                list_count = 0

                # while (list_count < result_length): change loop -> while loop to for loop
                for list_count in range(searchListLength): 
                    print("Crawling number = "+str(list_count+1)+"/"+str(searchListLength))
                #     titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a") ## 펼쳐보기
                    titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[2]/table/tbody/tr/td/div/div[4]/a") ## 간략보기
                    titleLinks[list_count].click()
                    time.sleep(2)

                    detailInfo = dict()
                    
                    # class="box_detail_point"
                    detailPoints = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[1]/*")
                    for detail in detailPoints:
                        detailInfo[detail.get_attribute("class")] = detail.text

                    # class="book_keyword"
                    keywords = []
                    if len(driver.find_elements(By.CLASS_NAME, "book_keyword")) > 0:
                        for keyword in driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[2]/div[4]/div[2]/*"):
                            keywords.append(keyword.text)
                        detailInfo['book_keyword'] = ", ".join(keywords)

                    # more button click
                    detailcontentList = []
                    contents = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[5]/div[1]/div[3]/*")

#                     case1. find all a tag / more button click
                    for content in contents:
                        if len(content.find_elements(By.TAG_NAME, "a"))>0:
                            for aTag in content.find_elements(By.TAG_NAME, "a"):
                                if aTag.get_attribute("class") == "btn_small btn_more" and aTag.text == "더보기":
                                    aTag.click()
                                    time.sleep(2)

#                     case2. contents css property check(display: none;) test
#                     for idx, content in enumerate(contents):
#                         print(len(content.find_elements(By.XPATH, ".//*")))
#                         print(content.tag_name + " | " + content.get_attribute("class"))

#                         if content.value_of_css_property("display") == "none" and content.tag_name!="script":
#                             print("none "+content.tag_name + " | " + content.get_attribute("class"))
#                             print("none "+content.tag_name + " | " + contents[idx-1].get_attribute("class"))

#                             print("before="+contents[idx-1].text)
#                             contents[idx-1].find_element(By.TAG_NAME, "a").click()
#                             print("after="+contents[idx-1].text)

                    # "content"
                    for content in contents:
                            detailcontentList.append(content.text)
                    detailInfo['content'] = "\n".join(detailcontentList)
                    # "URL"
                    detailInfo['url'] = driver.current_url
#                     print(detailInfo)
                    driver.back()
                #     list_count+=1
                    time.sleep(2)
                return detailInfo
            except Exception as e:
                print(e)

    else:
        exit()

In [3]:
bookSelection = BookSelection("http://www.kyobobook.co.kr/", "python")
driver = bookSelection.set_drivers()
bookSelection.get_url(driver)
bookSelection.get_searchKeywords(driver)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/administrator/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [4]:
searchTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[1]/ul").text.split('\n')
subTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[2]/ul").text.split('\n')
categoryList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[3]/table/tbody/tr[1]/td/ul").text.split('\n')

In [5]:
print(searchTabList)
print(subTabList)
print(categoryList)

['상품(5,299)', '중고장터(532)', '스토리K(25)', '리뷰(1,262)', '책소개.목차(3,400)']
['전체 5,299', '국내도서 1,157', '외국도서 3,605', 'eBook 500', 'sam 90', '음반/영상 37', '기프트 0']
['컴퓨터/IT 1,111', '어린이(초등) 11', '경제/경영 11', '과학 7', '청소년 3', '정치/사회 3', '취업/수험서 2', '자기계발 2']


In [6]:
## 리스트 보기 개수 조절
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]").click()
time.sleep(2)

driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]/ul/li[3]").click()
time.sleep(2)

## 리스트 간략히 보기
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[3]").click()
time.sleep(2)

driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[3]/ul/li[2]/a").click()
time.sleep(2)

totalPages = bookSelection.get_totalPage(driver)
print("totPageCnt = "+str(totalPages))

totPageCnt = 89


## 페이지 처리, 상세페이지 크롤링

In [8]:
for page in range(int(totalPages)):
    currentPage = bookSelection.get_currentPage(driver)
    print("CurrentPage = "+str(currentPage))
    
    ## Crawling List    
    searchList = bookSelection.get_searchList(driver)
    ## 상세페이지 Crawling
    detailInfo = bookSelection.crawling_detailPage(driver, searchList)
    
    if currentPage != totalPages:
        driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/a[3]").click()
    else:
        print("Complete crawling of all pages!!")
        break
    time.sleep(2)

CurrentPage = 1
Crawling number = 1/60
Crawling number = 2/60


KeyboardInterrupt: 